In [1]:
import pandas as pd
import math
import os.path
import time
import numpy as np
import talib
from talib import abstract
from binance.client import Client
from datetime import timedelta, datetime
from dateutil import parser
#backtest lib
from backtesting import Backtest
from backtesting.lib import SignalStrategy
from backtesting.lib import crossover

C:\ProgramData\Anaconda3\lib\site-packages\backtesting\_plotting.py:45: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
### API
binance_api_key = 'TESZdWxapwvZG90v1SJ3zMahcXySUkYv3bxRRKXKkfCMANtUS0Dw38iWALACQuHU'    #Enter your own API-key here
binance_api_secret = 'Uw1V3mbqoXDm9II7tC8lHRrYcGslHKhqo2bEXGelt08bnAn5lk4EbYfbKmHjWguY' #Enter your own API-secret here
### CONSTANTS ----add 15min
binsizes = {"1m": 1, "5m": 5,"15m":15, "1h": 60, "4h": 240, "1d": 1440}
batch_size = 750
# api instance
binance_client = Client(api_key=binance_api_key, api_secret=binance_api_secret)

In [3]:
### get data funcs
def minutes_of_new_data(symbol, kline_size, data, source):
    if len(data) > 0:  old = parser.parse(data["timestamp"].iloc[-1])
    elif source == "binance": old = datetime.strptime('1 Jan 2017', '%d %b %Y')
    elif source == "bitmex": old = bitmex_client.Trade.Trade_getBucketed(symbol=symbol, binSize=kline_size, count=1, reverse=False).result()[0][0]['timestamp']
    if source == "binance": new = pd.to_datetime(binance_client.get_klines(symbol=symbol, interval=kline_size)[-1][0], unit='ms')
    if source == "bitmex": new = bitmex_client.Trade.Trade_getBucketed(symbol=symbol, binSize=kline_size, count=1, reverse=True).result()[0][0]['timestamp']
    return old, new

def get_all_binance(symbol, kline_size, save = False):
    filename = '%s-%s-data.csv' % (symbol, kline_size)
    # jsaw edit: not to get data from API if aiready exist data
    if os.path.isfile(filename): 
        print("read exist")
        data_df = pd.read_csv(filename)
        return data_df
    else: 
        data_df = pd.DataFrame()
    oldest_point, newest_point = minutes_of_new_data(symbol, kline_size, data_df, source = "binance")
    delta_min = (newest_point - oldest_point).total_seconds()/60
    available_data = math.ceil(delta_min/binsizes[kline_size])
    if oldest_point == datetime.strptime('1 Jan 2017', '%d %b %Y'): 
        print('Downloading all available %s data for %s. Be patient..!' % (kline_size, symbol))
    else: 
        print('Downloading %d minutes of new data available for %s, i.e. %d instances of %s data.' % (delta_min, symbol, available_data, kline_size))
    klines = binance_client.get_historical_klines(symbol, kline_size, oldest_point.strftime("%d %b %Y %H:%M:%S"), newest_point.strftime("%d %b %Y %H:%M:%S"))
    data = pd.DataFrame(klines, columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore' ])
    data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms')
    if len(data_df) > 0:
        temp_df = pd.DataFrame(data)
        data_df = data_df.append(temp_df)
    else: 
        data_df = data
    data_df.set_index('timestamp', inplace=True)
    data_df = data.astype(float)
    if save: 
        data_df.to_csv(filename)
    print('All caught up..!')
    return data_df

In [4]:
#get data since 2017.7.1

df_4hr = get_all_binance('BTCUSDT', '4h',True)
df_1hr = get_all_binance('BTCUSDT', '1h',True)
df_15min = get_all_binance('BTCUSDT', '15m',True)
df_5min = get_all_binance('BTCUSDT', '5m', True)
df_1min = get_all_binance('BTCUSDT', '1m',True)

read exist
read exist
read exist
read exist
read exist


In [5]:
df = df_1hr
df

,timestamp,open,high,low,close,volume,close_time,quote_av,trades,tb_base_av,tb_quote_av,ignore
0,2019-11-01 00:00:00,9140.86,9144.70,9098.28,9125.52,1312.058126,1.572570e+12,1.196282e+07,16105.0,694.982113,6.336724e+06,0.0
1,2019-11-01 01:00:00,9127.00,9141.70,9090.08,9092.65,992.634124,1.572574e+12,9.048240e+06,17377.0,466.014431,4.248011e+06,0.0
2,2019-11-01 02:00:00,9092.71,9140.61,9062.09,9123.16,1379.955092,1.572577e+12,1.254906e+07,21772.0,685.744876,6.237552e+06,0.0
3,2019-11-01 03:00:00,9122.85,9122.85,9043.85,9063.13,1559.194143,1.572581e+12,1.414766e+07,17695.0,828.944292,7.522834e+06,0.0
4,2019-11-01 04:00:00,9064.12,9100.00,9030.00,9078.33,1155.043623,1.572584e+12,1.047422e+07,15400.0,671.470119,6.090970e+06,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9063,2020-11-13 07:00:00,16314.54,16314.54,16172.41,16253.30,2638.152564,1.605254e+12,4.285380e+07,42876.0,1288.647052,2.093242e+07,0.0
9064,2020-11-13 08:00:00,16253.31,16283.22,16122.33,16263.72,3458.247103,1.605258e+12,5.606513e+07,63311.0,1563.948723,2.535910e+07,0.0
9065,2020-11-13 09:00:00,16263.72,16357.86,16222.48,16324.61,3578.779497,1.605262e+12,5.831102e+07,63629.0,1824.216607,2.972066e+07,0.0
9066,2020-11-13 10:00:00,16324.60,16344.86,16165.00,16201.14,3377.321229,1.605265e+12,5.488366e+07,48153.0,1385.620443,2.251822e+07,0.0


In [32]:
class StrategyMACD(SignalStrategy):
    #optimize parameter
    
    #15min 24,20
    RSIperiod = 12
    RSICloseUpTrigger = 60
    RSICloseDownTrigger = 20
    RSIOpenTrigger = 50
    
    
    def init(self):
        super().init()
        
        #get strategy data
        df = self.getDf()
        #EMA
        df['EMA5'] = talib.EMA(df['close'], timeperiod=5)
        df['EMA10'] = talib.EMA(df['close'], timeperiod=10)
        df['EMA20'] = talib.EMA(df['close'], timeperiod=20)
        df['EMA60'] = talib.EMA(df['close'], timeperiod=60)
        df['EMA200'] = talib.EMA(df['close'], timeperiod=200)
        #MACD
        macdFast, macdSlow, histogram = talib.MACD(df['close'].values, fastperiod=12, slowperiod=26, signalperiod=9)
        df['MACD_DIF'] = macdFast
        df['MACD_DIM'] = macdSlow
        df['MACD_HIS'] = histogram
        #RSI
        df['RSI'] = talib.RSI(df['close'], timeperiod=self.RSIperiod)
        #print(df['RSI'])
        # cache to class for signal
        self.high = df['high']
        self.low = df['low']
        self.open = df['open']
        self.close = df['close']
        df['preClose'] = df['close'].shift(periods=1)
        df['preDIF'] = df['MACD_DIF'].shift(periods=1)
        df['preDIM'] = df['MACD_DIM'].shift(periods=1)

        signal_long = (self.close > df['EMA5']) & (df['EMA5'] > df['EMA10']) & (df['EMA10'] > df['EMA20']) & (df['EMA20'] > df['EMA60'])\
        & (df['EMA60'] > df['EMA200']) & (df['preClose'] < df['EMA5'])
        #print(signal_long)
        
        signal_closePosition = (df["RSI"] > self.RSICloseUpTrigger) & ( (df["MACD_DIF"] < df["MACD_DIM"]) \
                                                                       & (df["MACD_DIF"] < df["MACD_DIM"]) & (df['preDIF'] > df['preDIM']) )
        #signal = signal_long
        #signal[signal_short] = -1
        self.set_signal(signal_closePosition,signal_long)
        
    def getDf(self):
        df = pd.DataFrame(columns=['open','high', 'low', 'close','volume'])
        df['open'] = pd.Series(self.data.Open)
        df['close'] = pd.Series(self.data.Close)
        df['high'] = pd.Series(self.data.High)
        df['low'] = pd.Series(self.data.Low)
        return df
        
    def next(self):
        #if(self.position.pl_pct > 0.2):
        #    self.position.close()
        #if(self.position.pl_pct < -0.1):
        #    self.position.close()
            
        super().next()

In [31]:
backtestData = df.rename(columns={"open": "Open", "high": "High", "low": "Low", "close": "Close","volume":"Volume"})
backtestData = backtestData[["Open","High","Low","Close","Volume"]]
result1 = Backtest(backtestData, StrategyMACD, cash=100000, commission=.002)
print(result1.run())
result1.plot()

<ipython-input-31-005da5f7c270>:3: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  result1 = Backtest(backtestData, StrategyMACD, cash=100000, commission=.002)


Start                                       0
End                                      9067
Duration                                 9067
Exposure Time [%]                      9.0869
Equity Final [$]                      98237.7
Equity Peak [$]                        100073
Return [%]                           -1.76227
Buy & Hold Return [%]                  77.481
Max. Drawdown [%]                    -2.16831
Avg. Drawdown [%]                   -0.485453
Max. Drawdown Duration                   6317
Avg. Drawdown Duration                 1464.2
# Trades                                   37
Win Rate [%]                          43.2432
Best Trade [%]                        2.64364
Worst Trade [%]                      -7.98056
Avg. Trade [%]                      -0.456058
Max. Trade Duration                       307
Avg. Trade Duration                   22.0811
Profit Factor                        0.309713
Expectancy [%]                       0.832763
SQN                               

In [62]:
stats._strategy


<Strategy StrategyMACD(RSIperiod=12,RSICloseTrigger=72)>

In [61]:
#optimize
stats = result1.optimize(RSIperiod=range(5, 16, 1),
                         RSICloseTrigger=range(70, 95, 2),
                    maximize='Equity Final [$]')
stats

Start                                       0
End                                     36259
Duration                                36259
Exposure Time [%]                     46.6685
Equity Final [$]                       220554
Equity Peak [$]                        223657
Return [%]                            120.554
Buy & Hold Return [%]                 77.7434
Max. Drawdown [%]                    -17.3144
Avg. Drawdown [%]                     -1.6104
Max. Drawdown Duration                   5323
Avg. Drawdown Duration                184.661
# Trades                                   95
Win Rate [%]                          69.4737
Best Trade [%]                        30.3344
Worst Trade [%]                       -13.752
Avg. Trade [%]                        6.89922
Max. Trade Duration                      5253
Avg. Trade Duration                   2300.09
Profit Factor                         5.78792
Expectancy [%]                        10.7181
SQN                               